## Finding Cell-Cell Communication groups using scACCorDiON and CrossTalkeR

James S. Nagai

28.01.2024

*Note: Before using ACCorDIoN make sure that you have the CrossTalkeR installed in your local R enviroment([github](https://costalab.github.io/CrossTalkeR/))*

Here we introduce scACCorDIoN. In this notebook, we guide you into the framework steps.


In [1]:
import pandas as pd
import os 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pickle
from pydiffmap import diffusion_map as dm
from scaccordion import tl
import networkx as nx
from sklearn.preprocessing import Normalizer
import warnings
import ot
from scipy.sparse import diags
from scipy.sparse.linalg import eigs
from sklearn import manifold
from scipy.linalg import  norm
import phate
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import networkx as nx
import sci_palettes
from tqdm import tqdm
import kmedoids
import random
import itertools as it
np.random.seed(859)
warnings.filterwarnings("ignore")

/home/james/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
2025-01-28 11:40:58.381088: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory


In [2]:
f = open("../data/Distance_results/Peng_PDAC_processed_data_tmm_final.pickle",'rb')
metadata = pd.read_hdf(f"../data/metadata/Peng_PDAC_metaacc.h5ad")
Aaccdt = pickle.load(f)
labels=metadata.loc[Aaccdt.p.columns,'accLabel']

FileNotFoundError: [Errno 2] No such file or directory: '../data/Distance_results/Peng_PDAC_processed_data_tmm_final.pickle'

In [ ]:
weight="lr_means"
tmpcols = ['source','target',weight]
for k,v in Aaccdt.tbls.items():
    tmpvar = v.loc[:,tmpcols].groupby(['source','target']).sum().reset_index()
    prop = v.loc[:,tmpcols].groupby(['source','target']).size().reset_index()
    prop[0]=prop[0]/prop[0].sum()
    tmpvar['prop'] =  prop[0]
    Aaccdt.graphs[k]=nx.from_pandas_edgelist(tmpvar,
                                        edge_attr=[weight,'prop'],
                                        create_using=nx.DiGraph)
    Aaccdt.nodes.update(list(Aaccdt.graphs[k].nodes()))
Aaccdt.p = tl.utils.graphs_to_pmat(Aaccdt.nodes,Aaccdt.graphs,weight)
Aaccdt.p.sort_index(inplace=True)
c = tl.utils.graphs_to_pmat(Aaccdt.nodes,Aaccdt.graphs,'prop')
Aaccdt.c = c.loc[Aaccdt.p.index,:]

In [ ]:
Aaccdt.make_pca()

In [ ]:
pcaold = Aaccdt.Cs['PCA']

In [ ]:
Aaccdt.p = Aaccdt.c.apply(lambda x: x/np.sum(x))

In [ ]:
Aaccdt.make_pca()

In [ ]:
metadata.groupby("accLabel").size().reset_index()

In [ ]:
sns.scatterplot(x=Aaccdt.Cs['PCA'].loc[:,0],y=Aaccdt.Cs['PCA'].loc[:,1],hue=metadata['accLabel'])

In [ ]:
```{python}
km = kmedoids.KMedoids(n_clusters=3, method='fasterpam')
plt.figure(figsize=(6,5))
c = km.fit(Aaccdt.wdist['HTD_0.5'].to_numpy())
```

In [ ]:
X =np.round(Aaccdt.wdist['HTD_0.5'].loc[Aaccdt.wdist['HTD_0.5'].index,Aaccdt.wdist['HTD_0.5'].index],5)
clust,loss = [],[]
for _ in tqdm(range(100)):
    kmeans = tl.KBarycenters(k=3,init='++',random_state=42,max_iters=100)
    kmeans.fit(X,distr=Aaccdt.p.loc[list(Aaccdt.expgraph.nodes()),:],cost=Aaccdt.Cs['HTD_0.5'])
    clust.append(kmeans)
    loss.append(kmeans.centroids.mean().mean())
ncls= clust[np.argmin(loss)]

In [ ]:
aux = ncls.centroids.to_dict()
for i in range(3):
    for j in range(i,3):
        if i!=j:
            aux[i][j] = ot.emd2(a=ncls.bary[i]/ncls.bary[i].sum(),
                                b=ncls.bary[j]/ncls.bary[j].sum(),M=Aaccdt.Cs['HTD_0.5'])
            aux[j][i] = ot.emd2(a=ncls.bary[i]/ncls.bary[i].sum(),
                                b=ncls.bary[j]/ncls.bary[j].sum(),M=Aaccdt.Cs['HTD_0.5'])
        else:
            aux[i][j] = 0
eux = pd.concat([Aaccdt.wdist['HTD_0.5'],ncls.centroids],axis=1)
eux = pd.concat([eux,pd.DataFrame.from_dict(aux).T])
tmplab = ncls.flabels.tolist()
for i in [0,1,2]:
    tmplab.append(i) 

In [ ]:
tmpstl = ['0']*len(Aaccdt.p.columns)+['1']*3

In [ ]:
import phate

In [ ]:
emb1 = phate.PHATE(knn=8,knn_dist='precomputed',random_state=42)
emb1 = emb1.fit_transform(eux.to_numpy())

In [ ]:
%matplotlib agg
fs,axs = plt.subplots(1,2,figsize=(12,5))
axs = axs.ravel()

## Clustering

In [ ]:
barys = pd.DataFrame(ncls.bary,index=Aaccdt.p.loc[list(Aaccdt.expgraph.nodes()),:].index)
barys.reset_index(inplace=True)
barys[['u', 'v']] = barys['index'].str.split('$', expand=True)
cmapse =[plt.cm.Greens,plt.cm.Blues,plt.cm.Reds]
vtxcmap = {i[1]:plt.cm.tab10.colors[i[0]] for i in enumerate(Aaccdt.nodes)}

In [ ]:
sns.scatterplot(x=emb1[0:-3,0],
                y=emb1[0:-3,1],hue=labels,s=200,linewidth=0,ax=axs[0])
axs[0].legend(loc=0, bbox_to_anchor=(1, 0.5))

axs[0].invert_xaxis()

sns.scatterplot(x=emb1[:,0],
                y=emb1[:,1],hue=tmplab,s=200,style=tmpstl,palette=list(plt.cm.Set1.colors)[0:3],
                size=tmpstl,sizes=(400,200),linewidth=0,ax=axs[1])
axs[1].legend(loc=0, bbox_to_anchor=(1, 0.5))
axs[1].invert_xaxis()
fs.tight_layout()
fs.savefig("../figs/selection/PDAC_MDS.pdf")
fs

In [ ]:
fs,axs = plt.subplots(1,3,figsize=(18,5),dpi=72)
axs = axs.ravel()
cnt=0
for i in [2,1,0]:
    barynet =nx.from_pandas_edgelist(barys[barys[i]>1e-5].sort_values(i,ascending=True),
                            source='u',
                            target='v',
                            edge_attr=i,
                            create_using=nx.DiGraph)
    tempnet = nx.complete_graph(len(Aaccdt.nodes),create_using=nx.DiGraph)
    pos = nx.circular_layout(tempnet,scale=2)
    pos = {list(Aaccdt.nodes)[i[0]]:i[1] for i in enumerate(pos.values())}
    for id1 in Aaccdt.nodes:
        if  id1 not in barynet.nodes():
            barynet.add_node(id1)
    plt.figure(figsize=(5,5))
    currcmap = [vtxcmap[u] for u in barynet.nodes()]
    nsz=[i*5000 for i in list(nx.pagerank(barynet).values())]
    nx.draw_networkx_nodes(barynet,pos,node_size=nsz,node_color=currcmap,ax=axs[cnt])
    edcol = nx.get_edge_attributes(barynet,i)
    nx.draw_networkx_edges(barynet,pos=pos,edge_color=edcol.values(),
                           connectionstyle="arc3,rad=0.30",
                           edge_cmap=cmapse[::-1][i],width=2,ax=axs[cnt],arrowsize=40)
    for k,v in pos.items():
        v[1]+=np.sign(v[1])*0.1
        v[0]+=np.sign(v[0])*0.2
    nx.draw_networkx_labels(barynet,pos,verticalalignment='bottom',
                            font_size=18,clip_on=False,ax=axs[cnt])
    axs[cnt].axis('off')
    cnt+=1
    #plt.savefig(f'bary_{i}.pdf')
fs.tight_layout()
fs
fs.savefig("../../Accordion_Experiments/Revision_Figure_Folder/ccis.pdf")

In [ ]:
#tmaps.to_csv("../PDAC_Resuls_CC/tmaps.csv")

In [ ]:
f,axs1 = plt.subplots(1,1,figsize=(6 ,5))
sns.regplot(x="ctr->PDAC1",y="PDAC1->PDAC2",data=tmaps,fit_reg = False,
           x_jitter = 0.001, y_jitter = 0.001, scatter_kws = {'alpha' : 1,'s':80},ax=axs1)
n=10
ssource = tmaps.loc[tmaps['source1']==tmaps['source2'],:].sort_values("ctr->PDAC1",ascending=False)
starget =  tmaps.loc[tmaps['target1']==tmaps['target2'],:].sort_values("PDAC1->PDAC2",ascending=False)

def ann(row):
    r = row[1]
    ind = r["index"]
    axs1.annotate(ind, xy=(r["ctr->PDAC1"], r["PDAC1->PDAC2"]), 
            xytext=(0.1,0.1) , textcoords ="offset points")

for row in ssource.iterrows():
    ann(row)
for row in starget.iterrows():
    ann(row)
axs1.set_xlabel('CONTROL1 -> PDAC1')
axs1.set_ylabel('PDAC1 -> PDAC2')
f.savefig('../figs/tmap1.pdf')

In [ ]:
tmaps['selected'] = False

In [ ]:
tmaps['selected'][starget.index[0]] = True

In [ ]:
tmaps['selected'][ssource.index[6]] = True

In [ ]:
f,axs1 = plt.subplots(1,1,figsize=(6 ,5))
sns.regplot(x="ctr->PDAC1",y="PDAC1->PDAC2",data=tmaps,fit_reg = False,
           x_jitter = 0.001, y_jitter = 0.001, scatter_kws = {'alpha' : 1,'s':80},ax=axs1)
n=10
ssource = tmaps.loc[tmaps['source1']==tmaps['source2'],:].sort_values("ctr->PDAC1",ascending=False)
starget =  tmaps.loc[tmaps['target1']==tmaps['target2'],:].sort_values("PDAC1->PDAC2",ascending=False)

def ann(row):
    r = row[1]
    ind = r["index"]
    if r['selected']:
        axs1.annotate(ind, xy=(r["ctr->PDAC1"], r["PDAC1->PDAC2"]), 
                xytext=(0.1,0.1) , textcoords ="offset points")

for row in ssource.iterrows():
    ann(row)
for row in starget.iterrows():
    ann(row)
axs1.set_xlabel('CONTROL1 -> PDAC1')
axs1.set_ylabel('PDAC1 -> PDAC2')
f.savefig('../figs/tmap1.pdf')

In [ ]:
for i in ssource.iterrows():
    plt.figure()
    sns.scatterplot(x=i[1]['edge1'],y=i[1]['edge2'],
                    data=Aaccdt.p.T,hue=tmplab[0:-3],
                    palette=list(plt.cm.Set1.colors)[0:3],s=200,linewidth=0)
    plt.suptitle('CTR->PDAC1 to PDAC1->PDAC2')
    plt.savefig(f"../figs/same_source_1-{i[1]['index']}.pdf")
for i in starget.iterrows():
    plt.figure()
    sns.scatterplot(x=i[1]['edge1'],y=i[1]['edge2'],
                    data=Aaccdt.p.T,hue=tmplab[0:-3],
                    palette=list(plt.cm.Set1.colors)[0:3],s=200,linewidth=0)
    plt.suptitle('CTR->PDAC1 to PDAC1->PDAC2')
    plt.savefig(f"../figs/same_target_1-{i[1]['index']}.pdf")

In [ ]:
ncls.flabels

In [ ]:
lab = ['PDAC2',"PDAC1","CONTROL"]

In [ ]:
nlabels = {k:lab[v] for k,v in zip(Aaccdt.wdist['HTD_0.5'].index,ncls.flabels)}

In [ ]:
import scanpy as sc

In [ ]:
scadata = sc.read_h5ad("/home/james/icg/scRNA/Patients_scRNA/data/singlecell_processed/Peng_PDAC_processed.h5ad")

In [ ]:
scadata.obs['kbary'] = [nlabels[i] for i in scadata.obs['orig.ident']]

In [ ]:
scadata.obs.groupby(['kbary','accLabel']).size()

In [ ]:
scadata.obs.groupby(['kbary','accLabel1']).size()

In [ ]:
scadata.write_h5ad('PDAC_w_bary.h5ad')

# 

In [ ]:
sc.pl.umap(scadata,color='kbary')

In [ ]:
import liana as li

In [ ]:
if not os.path.exists("../data/PDAC"):
    os.mkdir("../data/PDAC")

In [ ]:
for i in lab:
    lr=li.method.cellphonedb(scadata[scadata.obs.kbary==i], 
                                  groupby='accAnnot', 
                                  expr_prop=0.15, 
                                  verbose=False,
                                  resource_name='consensus',inplace=False,use_raw=False)
    lr.to_csv(f"../data/PDAC/lr_{i}.csv")


In [ ]:
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
import rpy2.robjects.packages as rpackages
from rpy2.robjects import pandas2ri
#utils = rpackages.importr('utils')
#devtools = utils.install_packages('devtools')
d = {'package.dependencies': 'package_dot_dependencies',
     'package_dependencies': 'package_uscore_dependencies'}
ctker = importr('CrossTalkeR', 
           robject_translations = d)
data = {}
for i in os.listdir('../data/PDAC/'):
    if i.startswith('lr') and i.endswith('csv'):
        evfull = pd.read_csv('../data/PDAC/'+i)
        evfull = evfull.loc[:,['ligand','receptor','source','target','lr_means','cellphone_pvals']]
        evfull['type_gene_A'] = 'Ligand'
        evfull['type_gene_B'] = 'Receptor'
        evfull['gene_A'] = evfull['ligand']
        evfull['gene_B'] = evfull['receptor']
        evfull['MeanLR'] = evfull['lr_means']
        k=i[i.find('|')+1:len(i)-4]
        evfull.loc[list(evfull.cellphone_pvals.to_numpy()<=0.01),:].to_csv(f'../data/PDAC/ct_{i}')
        data[k]=os.path.abspath(f'../data/PDAC/ct_{i}')
an = ro.ListVector(data)

In [ ]:
if not os.path.exists("../data/PDAC/CTRL+"):
    os.mkdir("../data/PDAC/CTRL+")

In [ ]:

aux = ctker.generate_report(lrpaths=an.rx(ro.r("c('lr_CONTROL','lr_PDAC1','lr_PDAC2')")),
                out_path="/home/james/sciebo/Zenodo_scACCordion/data/PDAC/CTRL+",threshold = 0,
                out_file = 'control.html',
                output_fmt = "html_document",
                report=True,
                org = 'hsa')


In [ ]:
if not os.path.exists("../data/PDAC/PDAC1vsPDAC2"):
    os.mkdir("../data/PDAC/PDAC1vsPDAC2")

In [ ]:
aux = ctker.generate_report(lrpaths=an.rx(ro.r("c('lr_PDAC2','lr_PDAC1')")),
                out_path="/home/james/sciebo/Zenodo_scACCordion/data/PDAC/PDAC1vsPDAC2",
                            threshold = 0,
                out_file = 'control.html',
                output_fmt = "html_document",
                report=True,
                org = 'hsa')

In [ ]:
%load_ext rpy2.ipython


In [ ]:
%%R
require(CrossTalkeR)
require(ggplot2)
ctrpdac1 <- readRDS("../data/PDAC/CTRL+/LR_data_final.Rds")
ctrpdac2 <- readRDS("../data/PDAC/PDAC1vsPDAC2/LR_data_final.Rds")


In [ ]:
%%R
p1<-plot_sankey(ctrpdac1@tables$lr_PDAC1_x_lr_CONTROL,ligand_cluster='Ductal cell type 2',
                                               receptor_cluster="Ductal cell type 1",threshold=30)+
                ggtitle("CTR->PDAC1")
p2<-plot_sankey(ctrpdac1@tables$lr_PDAC2_x_lr_CONTROL,ligand_cluster='Ductal cell type 2',
                                               receptor_cluster="Ductal cell type 1",threshold=30)+
                ggtitle("CTR->PDAC2")
p3<-plot_sankey(ctrpdac2@tables$lr_PDAC1_x_lr_PDAC2,ligand_cluster='Ductal cell type 2',
                                               receptor_cluster="Ductal cell type 1",threshold=30)+
                ggtitle("PDAC2->PDAC1")
p1+p2+p3
ggsave("../figs/sankey/DCT22DCT1.pdf",width=25,height=7)


In [ ]:
%%R

write.csv(ctrpdac2@tables$lr_PDAC1_x_lr_PDAC2, "PDAC_differential.csv")

In [ ]:
%%R
p1<-plot_sankey(ctrpdac1@tables$lr_PDAC1_x_lr_CONTROL,ligand_cluster='Macrophage cell',
                                               receptor_cluster="Acinar cell",threshold=20)+
                ggtitle("CTR->PDAC1")
p2<-plot_sankey(ctrpdac1@tables$lr_PDAC2_x_lr_CONTROL,ligand_cluster='Macrophage cell',
                                               receptor_cluster="Acinar cell",threshold=20)+
                ggtitle("CTR->PDAC2")

p1+p2
ggsave("../figs/sankey/Mac2Acinar_top5.pdf",width=25,height=7)

In [ ]:
%%R

pdac1 <- ctrpdac2@tables$lr_PDAC1 |>
    dplyr::filter(cellpair %in% c("Ductal cell type 2@Ductal cell type 1")) |>
    dplyr::mutate(lrpair=paste(ligand,receptor,sep="@")) |>
    dplyr::select(lrpair, LRScore)
pdac2 <- ctrpdac2@tables$lr_PDAC2 |>
    dplyr::filter(cellpair %in% c("Ductal cell type 2@Ductal cell type 1")) |>
    dplyr::mutate(lrpair=paste(ligand,receptor,sep="@")) |>
    dplyr::select(lrpair, LRScore)

finaldf<-merge(pdac1,pdac2,by='lrpair',all=T)
finaldf[is.na(finaldf)]<-0
colnames(finaldf)<-c("LRpair","PDAC1","PDAC2")
write.csv(finaldf,"../data/PDAC_pairs.csv")

In [ ]:
import liana as li
import omnipath as op
import decoupler as dc

In [ ]:
net = dc.get_progeny(organism='human', top=5000)
lr_pairs = li.resource.select_resource('consensus')
lr_progeny = li.rs.generate_lr_geneset(lr_pairs, net, lr_sep="@")
lr_progeny.head()
lr = pd.read_csv("../data/PDAC_pairs.csv",index_col=0)

In [ ]:
lr.set_index("LRpair", inplace=True)
lr=lr.loc[lr.index.duplicated()!=True,:]

In [ ]:
estimate, pvals = dc.run_ulm(lr.transpose(), lr_progeny, source="source", target="interaction", use_raw=False)

In [ ]:
dc.plot_barplot(estimate, 'PDAC1', vertical=True, cmap='coolwarm', vmin=-7, vmax=7)
plt.tight_layout()
plt.title('PDAC1')
plt.savefig("../../Accordion_Experiments/Revision_Figure_Folder/PDAC1_ctker.pdf")

In [ ]:
dc.plot_barplot(estimate, 'PDAC2', vertical=True, cmap='coolwarm', vmin=-7, vmax=7)
plt.tight_layout()
plt.title('PDAC2')
plt.savefig("../../Accordion_Experiments/Revision_Figure_Folder/PDAC2_ctker.pdf")

In [ ]:
sc.pl.violin(scadata[scadata.obs.accAnnot=='Ductal cell type 2'],keys=['MMP7'],
             groupby='kbary',stripplot=False,save=True)
#plt.savefig('../PDAC_Resuls_CC/violin1.pdf')

In [ ]:
sc.pl.violin(scadata[scadata.obs.accAnnot=='Ductal cell type 1'],keys=['SDC1'],
             groupby='kbary',stripplot=False,save=True)

In [ ]:
p1=sns.scatterplot(x="PDAC1",y="PDAC2",data=estimate.T)
for line in range(0,estimate.T.shape[0]):
     p1.text(estimate.T["PDAC1"][line]+0.01, estimate.T["PDAC2"][line], 
     estimate.T.index[line], horizontalalignment='left', 
     size='medium', color='black')
plt.axline((-3,-3),(3,3),linestyle='--')
plt.xlim(-2,3)
plt.ylim(-2,3)
plt.title("Progeny Pathway Activity Score")
plt.tight_layout()
plt.savefig("../../Accordion_Experiments/Revision_Figure_Folder/PDAC_progeny.pdf")

In [ ]:
lr_progeny[lr_progeny.interaction.str.startswith("MMP7")].sort_values("weight",ascending=False).groupby("source").size()

In [ ]:
lr_progeny[lr_progeny.interaction.str.endswith("SDC1")].sort_values("weight",ascending=False).groupby("source").size()

In [ ]:
lr_progeny[lr_progeny.interaction.str.endswith("CD44")].sort_values("weight",ascending=False).groupby("source").size()

In [ ]:
lr_progeny[lr_progeny.interaction.str.startswith("SPP1")].sort_values("weight",ascending=False).groupby("source").size()